# Préparation des données

In [1]:
%run '2 - Récupération.ipynb'

## 1 - Inventaire des données

Après la réunion avec Selim, il nous a donné une marche à suivre très précise qui est détaillé dans le fichier [Rapport](../rapports/Rapport_2411.md). La première étape est de faire un inventaire des variables et de créer une base de donnée les contenant tous. Il faut bien garder en tête que les différents modèles ne verront que ligne par ligne.


### Variables qualitatives

   Nom de la variable    | Abréviation
   ----------------------|:-----------:
   Identifiant du joueur |  PLAYER_ID
   Équipe du joueur      |    TEAM
   Équipe adverse        |     ADV
   Domicile / Exterieur  |    HOME
   Adversaire directe    |   ADV_ID
 
### Variables quantitiatives

#### Variables scores positives

   Nom de la variable               | Abréviation 
   ---------------------------------|:-----------:
   Nombre de points                 |     PTS    
   Nombre de rebonds                |     REB
   Nombre de passes décisives       |     AST
   Nombre de récupération de balle  |     STL
   Nombre de block                  |     BLK
   Nombre de shoot réussi           |     FGM
   Nombre de lancer franc réussi    |     FTM
   Nombre de tire à 3 points réussi |     FG3M

#### Variables scores négatives

   Nom de la variable                | Abréviation 
   ----------------------------------|:-----------:
   Nombre de perte de balle          |     TOV
   Nombre de tire raté               |     FGF
   Nombre de lancer franc raté       |     FTF
   Nombre de tire à 3 points de raté |     FG3F

#### Variables de scores globales

   Nom de la variable            | Abréviation
   ------------------------------|:-----------:
   Somme des variables positives |    BONUS
   Somme des variables négatives |    MALUS
   Score du joueur               |    SCORE   
   
#### Variables laggées   
   
Toutes les variables ci-dessus ne sont pas accéssible pour le temps $t$ donc nous utilisons ces variables :
   
   Nom de la variable                                  |      Abréviation
   ----------------------------------------------------|:--------------------:
   Moyenne des N derniers matchs                       |   AVG_[ABREVIATION]
   Moyenne des N derniers matchs contre l'équipe [ADV] | AVG_ADV\\_[ABREVIATION]
   La série de victoire                                |        NB_VIC 
   La série de victoire contre l'équipe [ADV]          |       NB_VIC\\_ADV
   
### Variables temporelles

   Nom de la variable | Abréviation
   -------------------|:-----------:
   Date du match      |  GAME_DATE
   
### Hyperparamètres

   Nom du paramètre                                                     | Abréviation
   ---------------------------------------------------------------------|:-----------:
   Le nombre de match que l'on retient pour les moyennes                |      N
   Le nombre de match que l'on retient pour les moyennes par adversaire |    N_ADD


In [2]:
SEASON = 2019

# Hyperparamètre : le nombre de match que l'on regarde en arrière
N = 3
N_ADD = 1

# On récupère les données que l'on a récupéré précédemment
df = score_df(pd.read_csv(f'../data/boxscore_{SEASON}.csv'))

In [3]:
# La liste des variables dont on va récupérer les moyennes
list_var_avg = ['PTS', 'REB', 'AST', 'STL', 'BLK', 'FGM', 'FTM', 'FG3M',
                 'TOV', 'FGF', 'FG3F', 'FTF']

In [4]:
def avg_df(df, n=N):
    '''Calcul les moyennes en utilisant une 'rolling window' sur les variables 
    intéressantes
    
    Args :
        df : database avec les variables dans list_var_avg
        
    '''
    
    def avg(df):
        return df[list_var_avg].shift(periods=1)  \
                                .rolling(n)  \
                                .mean()  \
                                .add_prefix('AVG_')
                                
    
    # Calcul des moyennes pour chaque joueur puis on recréer une base
    df_mean = df.groupby('PLAYER_ID').apply(avg)  \
                                        .reset_index()  \
                                        .set_index('level_1')  \

    return pd.concat([df, df_mean.drop('PLAYER_ID', axis=1)], axis=1)

In [5]:
avg_df(df).sample(3)

,SEASON_ID,PLAYER_ID,GAME_ID,GAME_DATE,TEAM,ADV,HOME,WIN,PTS,REB,AST,STL,BLK,FGM,FTM,FG3M,TOV,FGF,FG3F,FTF,BONUS,MALUS,SCORE,AVG_PTS,AVG_REB,AVG_AST,AVG_STL,AVG_BLK,AVG_FGM,AVG_FTM,AVG_FG3M,AVG_TOV,AVG_FGF,AVG_FG3F,AVG_FTF
21727,22019,1629234,21901264,2020-08-05,SAS,DEN,True,False,12,5,1,1,0,6,0,0,1,2,0,0,25,3,22,5.333333,5.000000,0.666667,0.000000,1.333333,2.000000,1.333333,0.000000,2.000000,1.333333,0.000000,0.333333
7588,22019,203473,21900153,2019-11-12,SAC,POR,True,True,2,3,0,0,1,1,0,0,1,3,0,0,7,4,3,4.000000,4.666667,1.000000,0.333333,0.333333,1.000000,1.333333,0.666667,0.666667,2.666667,1.666667,0.000000
13236,22019,1627739,21900311,2019-12-04,CHI,MEM,True,True,10,4,4,1,0,4,0,2,2,3,2,0,25,7,18,5.000000,4.333333,3.000000,1.666667,0.000000,2.333333,0.000000,0.333333,2.000000,4.333333,2.000000,0.000000


In [6]:
def avg_add_df(df, n=N_ADD):
    # Calcul les moyennes en utilisant une 'rolling window' sur les variables 
    # intéressantes
    def avg_add(df):
        return df[list_var_avg].shift(periods=1)  \
                                .rolling(n)  \
                                .mean()  \
                                .add_prefix('AVG_ADD_')
    
    # Calcul des moyennes pour chaque joueur et pour chaque équipe 
    # puis on recréer une base
    df_add_mean = df.groupby('PLAYER_ID').apply(lambda x:
                                                x.groupby('ADV').apply(avg_add))  \
                                            .reset_index()  \
                                            .set_index('level_2')  \
    
    return pd.concat([df, df_add_mean.drop(['PLAYER_ID', 'ADV'], axis=1)]  \
                                                                      , axis=1)

In [7]:
# TRÈS LONG !!!
#avg_add_df(df).sample(3)

In [8]:
def nb_vic_df(df):
    
    # Code non vectoriel : 
    def nb_vic(df):
        # On calcul la serie de victoire jusqu'à une défaite
        serie = [0]
        for i in range(1, len(df)):
            serie += [serie[-1] + 1] if df.WIN.iloc[i-1] else [0]

        return pd.Series(serie, index=df.index, name='NB_VIC').to_frame()

    # On calcul les séries pour chaque joueur
    df_nb_vic = df.groupby('PLAYER_ID').apply(nb_vic)

    # On rajoute les séries à la fin de notre dataframe
    return pd.concat([df, df_nb_vic], axis=1)

In [9]:
nb_vic_df(df).sample(3)

,SEASON_ID,PLAYER_ID,GAME_ID,GAME_DATE,TEAM,ADV,HOME,WIN,PTS,REB,AST,STL,BLK,FGM,FTM,FG3M,TOV,FGF,FG3F,FTF,BONUS,MALUS,SCORE,NB_VIC
7738,22019,203482,21900880,2020-02-28,MIA,DAL,True,True,13,7,9,0,0,5,0,3,1,3,1,2,37,7,30,0
15832,22019,1628372,21900659,2020-01-22,NYK,LAL,True,False,0,1,0,0,0,0,0,0,1,2,0,0,1,3,-2,0
3143,22019,201952,21900349,2019-12-09,MIN,PHX,False,False,16,3,4,2,0,6,3,1,3,8,1,1,35,13,22,0


In [10]:
def nb_vic_add_df(df):
    
    # Code non vectoriel : 
    def nb_vic_add(df):

        # On calcul la serie de victoire jusqu'à une défaite
        serie = [0]
        for i in range(1, len(df)):
            serie += [serie[-1] + 1] if df.WIN.iloc[i-1] else [0]
        return pd.Series(serie, index=df.index, name='NB_VIC_ADD').to_frame()

    # On calcul les séries pour chaque joueur et pour chaque équipe
    df_nb_vic_add = df.groupby('PLAYER_ID').apply(lambda df: 
                        df.groupby('ADV').apply(nb_vic_add))  \
                                            .reset_index()  \
                                            .set_index('level_1')  \
                                            .drop('PLAYER_ID', axis=1)

    return pd.concat([df, df_nb_vic_add], axis=1)

In [11]:
# TRÈS LONG !!
#nb_vic_add_df(df)

In [12]:
# TRÈS LONG !!!
# La dataframe finale  

#df = nb_vic_add_df(nb_vic_df(avg_add_df(avg_df(df))))
#df.to_csv('../data/boxscore_2019_final.csv', index=False)

pd.read_csv('../data/boxscore_2019_final.csv')

,SEASON_ID,PLAYER_ID,GAME_ID,GAME_DATE,TEAM,ADV,HOME,WIN,PTS,REB,AST,STL,BLK,FGM,FTM,FG3M,TOV,FGF,FG3F,FTF,BONUS,MALUS,SCORE,AVG_PTS,AVG_REB,AVG_AST,AVG_STL,AVG_BLK,AVG_FGM,AVG_FTM,AVG_FG3M,AVG_TOV,AVG_FGF,AVG_FG3F,AVG_FTF,AVG_BONUS,AVG_MALUS,AVG_SCORE,AVG_ADD_PTS,AVG_ADD_REB,AVG_ADD_AST,AVG_ADD_STL,AVG_ADD_BLK,AVG_ADD_FGM,AVG_ADD_FTM,AVG_ADD_FG3M,AVG_ADD_TOV,AVG_ADD_FGF,AVG_ADD_FG3F,AVG_ADD_FTF,AVG_ADD_BONUS,AVG_ADD_MALUS,AVG_ADD_SCORE,NB_VIC,NB_VIC_ADD
0,22019,1713,21900014,2019-10-24,ATL,DET,False,True,0,0,1,0,0,0,0,0,1,2,2,0,1,5,-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,22019,1713,21900028,2019-10-26,ATL,ORL,True,True,2,2,1,1,0,1,0,0,0,2,2,0,7,4,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
2,22019,1713,21900043,2019-10-28,ATL,PHI,True,False,4,3,1,1,0,2,0,0,0,3,3,0,11,6,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0
3,22019,1713,21900052,2019-10-29,ATL,MIA,False,False,7,1,0,0,0,3,0,1,1,5,5,0,12,11,1,2.000000,1.666667,1.000000,0.666667,0.000000,1.000000,0.0,0.000000,0.333333,2.333333,2.333333,0.0,6.333333,5.000000,1.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,22019,1713,21900066,2019-10-31,ATL,MIA,True,False,2,1,0,0,0,1,0,0,0,4,3,0,4,7,-3,4.333333,2.000000,0.666667,0.666667,0.000000,2.000000,0.0,0.333333,0.333333,3.333333,3.333333,0.0,10.000000,7.000000,3.000000,7.0,1.0,0.0,0.0,0.0,3.0,0.0,1.0,1.0,5.0,5.0,0.0,12.0,11.0,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24044,42019,1629752,41900167,2020-09-01,UTA,DEN,False,False,1,1,0,0,0,0,1,0,0,2,2,3,3,7,-4,0.000000,2.333333,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,2.666667,0.000000,2.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
24045,22019,1629760,21900643,2020-01-20,CLE,NYK,True,False,2,1,1,0,1,1,0,0,0,1,1,0,6,2,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
24046,22019,1629760,21900700,2020-01-28,CLE,NOP,True,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
24047,22019,1629760,21900902,2020-03-02,CLE,UTA,True,False,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
